## Comencemos importando las bibliotecas que usaremos:

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
#Load data from csv

In [3]:
df = pd.read_csv('../data/raw/raw_1.csv', index_col=0)

In [4]:
df

,names,mass,distance,discovery_year,stellar_magnitude
0,11 Comae Berenices b,19.4 Jupiters,304.0,2007,4.72307
1,11 Ursae Minoris b,14.74 Jupiters,409.0,2009,5.01300
2,14 Andromedae b,4.8 Jupiters,246.0,2008,5.23133
3,14 Herculis b,4.66 Jupiters,58.0,2002,6.61935
4,16 Cygni B b,1.78 Jupiters,69.0,1996,6.21500
...,...,...,...,...,...
5039,XO-7 b,0.709 Jupiters,764.0,2019,10.52100
5040,YSES 2 b,6.3 Jupiters,357.0,2021,10.88500
5041,YZ Ceti b,0.7 Earths,12.0,2017,12.07400
5042,YZ Ceti c,1.14 Earths,12.0,2017,12.07400


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5044 entries, 0 to 5043
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   names              5044 non-null   object 
 1   mass               5044 non-null   object 
 2   distance           5028 non-null   float64
 3   discovery_year     5044 non-null   int64  
 4   stellar_magnitude  4927 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 236.4+ KB


In [6]:
df_2 = pd.read_csv("../data/raw/raw_2.csv", index_col=0)

In [7]:
df_2

,planet_type,radius,orbital_radius,orbital_days,eccentricity,detection_method
0,Gas Giant,1.08 x Jupiter(estimate),1.29,326.0,0.23,Radial Velocity
1,Gas Giant,1.09 x Jupiter(estimate),1.53,1.4,0.08,Radial Velocity
2,Gas Giant,1.15 x Jupiter(estimate),0.83,185.8,0.0,Radial Velocity
3,Gas Giant,1.15 x Jupiter(estimate),2.93,4.9,0.37,Radial Velocity
4,Gas Giant,1.2 x Jupiter(estimate),1.66,2.2,0.68,Radial Velocity
...,...,...,...,...,...,...
5039,Gas Giant,1.373 x Jupiter,0.04421,2.9,0.04,Transit
5040,Gas Giant,1.14 x Jupiter(estimate),115.0,1176.5,0.0,Direct Imaging
5041,Terrestrial,0.913 x Earth(estimate),0.01634,2.0,0.06,Radial Velocity
5042,Super Earth,1.05 x Earth(estimate),0.02156,3.1,0.0,Radial Velocity


## Transform Mass From jupiter to Earth

In [8]:
mass_E = []
for i in df.mass:
	try:
		if i.split(' ')[1] == 'Jupiters':
			J = float(i.split(' ')[0])
			mass_E.append(J*317.8)
		else:
			E = float(i.split(' ')[0])
			mass_E.append(E)
	except:
		mass_E.append(None)
df['mass_E'] = mass_E

# Merge Dataframes

In [9]:
df['orbital_days'] = df_2.orbital_days
df['detection_method'] = df_2.detection_method
df['planet_type'] = df_2.planet_type

### Transform Eccentricity to float

In [10]:
ecc = []
for i in df_2.eccentricity:
	try:
		ecc.append(float(i))
	except:
		ecc.append(None)
df['eccentricity'] = ecc

### Orbital Radius have Correlation of 1 with Orbital Days

In [11]:
# orb_rad = []
# for i in df_2.orbital_radius:
# 	if i == 'Unknown':
# 		orb_rad.append(None)
# 	else:
# 		orb_rad.append(float(i))
# df['orbital_radius'] = orb_rad

## Transform Radius From jupiter to Earth

In [12]:
rad = []
scale = []
for i in df_2.radius:
	if i == 'Unknown':
		rad.append(None)
		scale.append(None)
	else:
		temp = i.split(' ')
		if len(temp) == 3:
			rad.append(float(temp[0]))
			scale.append(temp[2])
		else:
			rad.append(float(temp[0]))
			scale.append(None)
df['radius'] = rad
df['scale_radius'] = scale

In [13]:
df_rad = df.loc[:,['radius','scale_radius']]
idx = []
n = 0
for i in df_rad.scale_radius:
	if i != None:
		if i[:7] == 'Jupiter':
			idx.append(n)
	n +=1

df_rad.radius[df_rad.index.isin(idx)] = df_rad.radius*11.2089
df['radius_E'] = df_rad.radius



C:\Users\Paris\AppData\Local\Temp\ipykernel_11040\1303421568.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rad.radius[df_rad.index.isin(idx)] = df_rad.radius*11.2089


### Drop Correlational Columns

In [14]:
df = df.drop(['mass','radius','scale_radius'], axis=1)

### Drop Unknowns

In [15]:
trash = df[df.planet_type == 'Unknown']
trash

,names,distance,discovery_year,stellar_magnitude,mass_E,orbital_days,detection_method,planet_type,eccentricity,radius_E
4364,KIC 10001893 b,5457.0,2014,15.829,NaN,0.2,Orbital Brightness Modulation,Unknown,0.0,NaN
4365,KIC 10001893 c,5457.0,2014,15.829,NaN,0.3,Orbital Brightness Modulation,Unknown,0.0,NaN
4366,KIC 10001893 d,5457.0,2014,15.829,NaN,0.8,Orbital Brightness Modulation,Unknown,0.0,NaN
4443,LkCa 15 b,516.0,2015,12.025,NaN,inf,Direct Imaging,Unknown,0.0,NaN
4444,LkCa 15 c,516.0,2015,12.025,NaN,inf,Direct Imaging,Unknown,0.0,NaN


In [16]:
df = df.drop(trash.index, axis=0)

### Save Categorical Columns

In [17]:
target = df.planet_type
names = df.names
detect =df.detection_method

# Encode Target

In [18]:
planet_encode = {
    'Gas Giant':3,
    'Neptune-like':2,
    'Super Earth':1,
    'Terrestrial':0
}

In [19]:
df.planet_type = df.planet_type.map(planet_encode)

### Impute Infinite

In [20]:
df[df.orbital_days == df.orbital_days.max()]

,names,distance,discovery_year,stellar_magnitude,mass_E,orbital_days,detection_method,planet_type,eccentricity,radius_E
120,CT Chamaleontis b,622.0,2007,12.335,5402.6,inf,Direct Imaging,3,0.0,24.65958


In [21]:
df.loc[120,['orbital_days']] = df.orbital_days.mode()[0]

# Fill Nan

In [22]:
# from sklearn.impute import SimpleImputer
# def MeanImpute(df):
#     for column in df.select_dtypes(np.number).columns:
#         df['column'] = SimpleImputer(strategy='mean')
#     return df


# df = MeanImpute(df)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5039 entries, 0 to 5043
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   names              5039 non-null   object 
 1   distance           5023 non-null   float64
 2   discovery_year     5039 non-null   int64  
 3   stellar_magnitude  4922 non-null   float64
 4   mass_E             5021 non-null   float64
 5   orbital_days       5039 non-null   float64
 6   detection_method   5039 non-null   object 
 7   planet_type        5039 non-null   int64  
 8   eccentricity       4862 non-null   float64
 9   radius_E           5030 non-null   float64
dtypes: float64(6), int64(2), object(2)
memory usage: 562.1+ KB


In [24]:
df.isna().sum()

names                  0
distance              16
discovery_year         0
stellar_magnitude    117
mass_E                18
orbital_days           0
detection_method       0
planet_type            0
eccentricity         177
radius_E               9
dtype: int64

In [25]:
df.isna().sum().sum()

337

In [26]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
df = pd.DataFrame(imputer.fit_transform(df.select_dtypes(np.number)),columns=df.select_dtypes(np.number).columns)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   distance           5039 non-null   float64
 1   discovery_year     5039 non-null   float64
 2   stellar_magnitude  5039 non-null   float64
 3   mass_E             5039 non-null   float64
 4   orbital_days       5039 non-null   float64
 5   planet_type        5039 non-null   float64
 6   eccentricity       5039 non-null   float64
 7   radius_E           5039 non-null   float64
dtypes: float64(8)
memory usage: 315.1 KB


# Log Transform Features

In [28]:
df['log_mass'] = np.log(df.mass_E)
df['log_distance'] = np.log(df.distance)
df['log_orbital'] = np.log(df.orbital_days)

In [29]:
log_radE = []
for i in df.radius_E:
	try:
		log_radE.append(np.log(i))
	except:
		log_radE.append(None)
df['log_radius'] = log_radE

### Save data

In [30]:
df.to_csv("../data/preprocessed/test.csv")